# Ordenando un toque:
    - Lo que buscamos es encontrar algún (o algunos) modelos que nos permitan aproximar alguna de las columnas del dataset (si, alguna).
    - Si o si tenemos que hacerlo co nla columna "precio", pero además tenemos que elgir otra (está en el enunciado).
    
    - Separemos los tantos:
        - Feature Engeniere es algo que vamos a hacer POR FUERA de todo lo que es la ejecución, es decir, cuando nosotros hagamos fit + predict + loquesea, esta nueva columna no tiene idea de que es nueva, asi que esto no nos importa tanto (para lo que es la ejecucion pura y dura)
       
        - Segmentación + K-fold + errores, es algo que nos interesa DENTRO de la ejecución.
        - Ejecutar 1 vez y tomar metricas, no estaría bueno, por lo que ya sabemos (overfitting), entonces no nos queda otra más que para cada segmento, realizar el KFold, callcular las métricas en cada Fold, y guardar el promedio de esas métricas (nos queda un array de metricas promedio en cada segmento).
        - ¿Donde empieza el prolema? En que para cada Fold, los valores de los factores de la ecuación de la recta (esos que obtenemos cuando hacemos fit) van a cambiar siempre, porque para cada "cacho" de dataset, estas cambian.
        - ¿Nos importa realmente? Obviamente googlie esto porque estaba medio perdido, y la respuesta es que no (a priori). Lo primero que a nosotros nos interesa saber, es si nuestro modelo "esta bueno", es decir, si el error total es bajo (comparando distintas métricas y demas). Esto no nos da las variables "finales", una vez que definimos que algún modelo funciona bien, recién ahí ejecutamos la regresión con TODOS los datos, y no quedamos con los factores resultantes (si usamos segmentación, seria un conjunto de factores para cada segmento, y para predecír deberíamos saber determinar a que segmento pertenece)
        
        
Esto lo saque de acá:
https://stats.stackexchange.com/questions/52274/how-to-choose-a-predictive-model-after-k-fold-cross-validation
https://stats.stackexchange.com/questions/2306/feature-selection-for-final-model-when-performing-cross-validation-in-machine?rq=1

Por si estoy entendiendo cualquier cosa.

In [1]:
import metnum
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from Model import Model
from Segment import Segment

In [2]:
df_original = pd.read_csv('../data/train.csv')
df = df_original.copy()

In [3]:
model1 = Model(df, ['metroscubiertos'], ['provincia', 'ciudad'])
segments = model1.regresionar()

RuntimeError: /Jalisco/Chiquilistlán tiene menos de 5 elementos

In [ ]:
model1.error_gral()

In [ ]:
for seg in model1.segments:
    print(seg.errors)

In [ ]:
df.info()